In [1]:
from openff.toolkit import Molecule

In [2]:
my_mol = Molecule.from_file("../polymer/HPMCAS-H-Trimer.sdf")
# Note there are no partial charges
print(my_mol.partial_charges)
# Now use NAGL to assign the partial charges
my_mol.assign_partial_charges(
    partial_charge_method="openff-gnn-am1bcc-1.0.0.pt"
)

None


In [3]:
my_mol.partial_charges

Magnitude,[0.4064335576384454 0.09678571439675383 0.10430543638161711 0.3341093114703088 -0.3985544689805121 0.12056351966551833 0.09279276199034743 0.1204054406969934 -0.5794452377469153 -0.41596194588252017 -0.3887450643689246 -0.42930790268488833 0.6533923200457483 -0.157144198647222 -0.530458743205747 0.11735256529740386 0.12987705088547757 0.13856901741913846 -0.11882373177119203 -0.5932327457577795 0.0652180365412622 0.1068299464075952 0.045508434542933025 0.07587677008799605 0.05800587452343992 0.04822633183888487 0.04822633183888487 0.08127689875058226 0.08127689875058226 0.08127689875058226 0.035808955797472515 0.035808955797472515 0.035808955797472515 0.043408611336269894 0.043408611336269894 0.048085933098116436 0.047268086591282406 0.047268086591282406 0.047268086591282406 0.3997451416342645 0.08478567517689757 0.09007818765334182 0.33471322573117307 -0.39938124500819155 0.11276411778620772 0.13345484949044278 0.13475877560071042 -0.3755863853604407 -0.4386221715123267 -0.37499671660967776 -0.37587955080576846 0.12620898104600003 0.11644070363930754 0.6474763802378565 -0.5459100552708716 -0.13374347471304843 -0.12380380415030427 0.6344623020975977 -0.5445361682087988 -0.6059879609257788 0.052147676715174236 0.06163774199179701 0.04685935891560606 0.06692780322722487 0.06581394887856536 0.07007228351525359 0.07007228351525359 0.0321674770801454 0.0321674770801454 0.0321674770801454 0.035423030623713055 0.035423030623713055 0.035423030623713055 0.10001028097561888 0.10001028097561888 0.10050888127974562 0.10050888127974562 0.44924867666653684 0.11243933952740721 0.07780416823319487 0.1065172678916841 -0.3854047902733893 0.0920589468567758 0.14736117936066678 0.13479409135273984 -0.37442245565958926 -0.443311447730741 -0.3810105272442908 -0.395957852235517 0.12130877889088683 0.11587183690957122 0.6419428638308435 -0.5394556471020788 -0.15218784891673037 0.06218593336037687 0.050219816633978405 0.062053100684204616 0.07588703698805861 0.06391566015175872 0.07077578968695693 0.07077578968695693 0.03320526100091032 0.03320526100091032 0.03320526100091032 0.03543472058466963 0.03543472058466963 0.03543472058466963 0.07868870116166167 0.07868870116166167 0.07868870116166167 0.062053130486527004]
Units,elementary_charge


In [4]:
my_mol.name

'HPMCAS-H-Trimer-polymer'

In [5]:
from openpharmmdflow.pipeline.sm.pipeline_settings import *
from openpharmmdflow.pipeline.sm.pipeline import SmallMoleculePipeline
import nglview

/home/mmh/micromamba/envs/openpharmmdflow-new-deps/lib/python3.13/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [6]:
settings = {
    # Not used right now
    "work_dir": "/home/mmh/Projects/OpenPharmMDFlow/experiments/sm/scratch",
    "inputs": [my_mol],
    "prep_config": SmallMoleculePipelinePrepConfig(
        bespokefit_config=BespokefitConfig(
            bespoke_workflow_factory_config=BespokeWorkflowFactoryConfig(),
            bespoke_executor_config=BespokeExecutorConfig(),
            mol_to_bespoke="ibuprofen",
        )
    ),
    "pack_config": SmallMoleculePipelinePackConfig(
        molecule_names=[
            my_mol.name,
        ],
        number_of_copies=[5],
        target_density=0.1,
    ),
    "parameterize_config": SmallMoleculePipelineParameterizeConfig(),
    "simulate_config": SmallMoleculePipelineSimulateConfig(),
    "analyze_config": SmallMoleculePipelineAnalyzeConfig(),
    "solvate_config": SmallMoleculePipelineSolvateConfig(
        nacl_conc=Quantity(0.1, "mole / liter"),
        padding=Quantity(1.2, "nanometer"),
        box_shape=RHOMBIC_DODECAHEDRON,
        target_density=Quantity(0.9, "gram / milliliter"),
        tolerance=Quantity(2.0, "angstrom"),
    ),
}
sm_config = SmallMoleculePipelineConfig(**settings)

In [7]:
smp = SmallMoleculePipeline(sm_config)
smp.load()

In [8]:
smp.pack()

Assigning residue names to molecules:

Verification (showing first 3 molecules):
  Molecule 0: HPMCAS-H-Trimer-polymer -> residue_name = HPM
  Molecule 1: HPMCAS-H-Trimer-polymer -> residue_name = HPM
  Molecule 2: HPMCAS-H-Trimer-polymer -> residue_name = HPM

Residue name assignment complete.



In [9]:
smp.topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

In [10]:
smp.solvate()

Assigning residue names to molecules:

Verification (showing first 3 molecules):
  Molecule 0: HPMCAS-H-Trimer-polymer -> residue_name = HPM
  Molecule 1: HPMCAS-H-Trimer-polymer -> residue_name = HPM
  Molecule 2: HPMCAS-H-Trimer-polymer -> residue_name = HPM
Assigning residue names to molecules:

Verification (showing first 3 molecules):
  Molecule 0: HPMCAS-H-Trimer-polymer -> residue_name = HPM
  Molecule 1: HPMCAS-H-Trimer-polymer -> residue_name = HPM
  Molecule 2: HPMCAS-H-Trimer-polymer -> residue_name = HPM

Residue name assignment complete.



In [11]:
smp.solvated_topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

In [12]:
smp.parameterize()

/home/mmh/micromamba/envs/openpharmmdflow-new-deps/lib/python3.13/site-packages/openff/interchange/operations/_combine.py:104: InterchangeCombinationWarning: Interchange object combination is complex and may produce strange results outside of use cases it has been tested in. Use with caution and thoroughly validate results!
  warnings.warn(
/home/mmh/micromamba/envs/openpharmmdflow-new-deps/lib/python3.13/site-packages/openff/interchange/operations/_combine.py:183: InterchangeCombinationWarning: Setting positions to None because one or both objects added together were missing positions.
  warnings.warn(


In [13]:
smp.interchange.visualize("nglview")

NGLWidget()

In [14]:
smp.simulate()

🚀 Setting up MD simulation...
📁 Setting up trajectory reporters in: .
   🔸 DCD trajectory: ./trajectory.dcd (every 500 steps)
   🔸 Thermodynamic data: ./simulation_data.csv (every 10 steps)
   🔸 Checkpoint: ./checkpoint.chk (every 5000 steps)
   ✅ HDF5 trajectory: ./trajectory.h5 (analysis-optimized)
   🔸 PDB visualization: ./trajectory_vis.pdb (every 50000 steps)
🚀 Starting simulation setup...
⚡ Minimizing energy...
✅ Energy minimization completed: PE = -117610.4 kJ/mol
🌡️ Initial velocities set to 300 K
🏃 Starting MD simulation: 5,000 steps
   Timestep: 1 fs
   Total time: 0.01 ns

Step,      Time (ps), Volume (nm³), Temperature (K), PE (kJ/mol)
-----------------------------------------------------------------
     250,       0.25,     87.875,        300.0,    -105836.5
     500,       0.50,     86.915,        300.0,    -104656.6
     750,       0.75,     86.524,        300.0,    -103304.5
    1000,       1.00,     85.646,        300.0,    -102312.1
    1250,       1.25,     85.065, 